In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## PROBLEMLER
1) Trafik akışının değişmesi kazaları nasıl etkiler?
2) Kaza oranlarını ne artırır?
3) Zaman içinde kaza oranlarını tahmin edebilir miyiz?
4) Kırsal ve kentsel alanlar nasıl farklılaştı?


## Sütun Bilgisi

* **Accident_Index:**	Her kazaya özel benzersiz bir kimlik numarası
* **Location_Easting_OSGR:**	OSGR koordinat sistemine göre doğu (x) yönündeki konum
* **Location_Northing_OSGR:**	OSGR koordinat sistemine göre kuzey (y) yönündeki konum
* **Longitude:**	Enlem (doğu/batı) konumu (coğrafi koordinat)
* **Latitude:**	Boylam (kuzey/güney) konumu (coğrafi koordinat)

🚨 KAZA DETAYLARI
* **Police_Force:** Kazayı kaydeden polis biriminin kodu
* **Accident_Severity:**	Kazanın ciddiyeti: Genellikle 1=Ölümcül, 2=Ciddi, 3=Hafif
* **Number_of_Vehicles:**	Kazaya karışan toplam araç sayısı
* **Number_of_Casualties:**	Toplam yaralı veya ölü kişi sayısı
* **Date:**	Kazanın gerçekleştiği tarih
* **Day_of_Week:**	Kazanın hangi gün olduğuna dair bilgi (1=Sunday, 2=Monday...)
Time	Kazanın gerçekleştiği saat

🗺️ LOKASYON BİLGİLERİ
**Local_Authority_(District):**	İlçe düzeyinde yerel idari birim
**Local_Authority_(Highway):**	Yol yönetiminden sorumlu yerel otorite

🛣️ YOL BİLGİLERİ
* **1st_Road_Class**	Ana yolun sınıfı (örneğin A road, B road, vb.)
* **1st_Road_Number**	Ana yolun numarası (örneğin A406 gibi)
* **Road_Type**	Yolun tipi: tek yön, çift yön, döner kavşak vb.
* **Speed_limit**	Yolun yasal hız sınırı (km/s veya mph)

🔀 KAVŞAK & YAN YOL BİLGİLERİ
* **Junction_Detail**	Kavşak türü: T-kavşak, Y-kavşak, döner kavşak vb.
* **Junction_Control**	Kavşakta ışık, tabela veya polis kontrolü olup olmadığı

| 2nd_Road_Class | İkinci yolun (yan yol) sınıfı |
| 2nd_Road_Number | İkinci yolun numarası |

🚶 YAYALAR & GEÇİTLER
* **Pedestrian_Crossing-Human_Control:**	Yaya geçidinde polis veya görevli kontrolü var mı?
* **Pedestrian_Crossing-Physical_Facilities**	Fiziksel yaya geçidi (ışık, çizgi, üst geçit vb.) var mı?

🌤️ ÇEVRE KOŞULLARI
* **Light_Conditions**	Aydınlatma durumu (gündüz, gece, sokak lambası vs.)
* **Weather_Conditions**	Hava durumu: yağmur, kar, açık hava vb.
* **Road_Surface_Conditions**	Yol yüzeyi: kuru, ıslak, buzlu vb.

⚠️ ÖZEL DURUMLAR & RİSKLER
 * **Special_Conditions_at_Site**	Geçici yol çalışması, bayrak, işaret vb. özel durumlar
* **Carriageway_Hazards**	Yolda tehlike var mı? (örneğin: döküntü, hayvan, araç vb.)

🏙️ ALAN & POLİS DURUMU
* **Urban_or_Rural_Area**	Kaza kentsel (şehir içi) mi yoksa kırsal bölgede mi?
* **Did_Police_Officer_Attend_Scene_of_Accident**	Kaza yerine polis geldi mi? (Evet/Hayır)

🧭 BÖLGESEL / YIL
* **LSOA_of_Accident_Location**	İngiltere’de bölgesel analiz için kullanılan "Lower Layer Super Output Area" kodu (çok küçük idari bölge)
Year	Kazanın gerçekleştiği yıl

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [ ]:
df1 = pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2005_to_2007.csv')
df2 = pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2009_to_2011.csv')
df3 = pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv')

In [ ]:
df = pd.concat([df1,df2,df3],ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

## İlk problem : *Trafik akışının değişmesi kazaları nasıl etkiler?*
*Trafik Akışını etkileyen faktörler:* Road_Type,Speed_Limit,Time,Weather_Conditions,Urban_or_Rural_Area, Junction_Detail(Çok fazla eksik değer var kullanılmayacak),Junction_Control(Çok fazla eksik değer var kullanılmayacak)

### Her faktöre göre kaza sayısı dağılımı:

In [ ]:
# Hız sınırına göre kazalar:
df.groupby("Speed_limit")["Accident_Index"].nunique().plot(kind="bar",title="Speed Limit  vs Accident Count")

#Speed Limit = 30 olduğunda maximum kaza sayısı. sonrasında 60 geliyor
# Demek ki şehir içinde kaza sayısı daha fazla

In [ ]:
#Yol türüne göre kazalar:
df.groupby('Road_Type')['Accident_Index'].nunique().sort_values().plot(kind="barh",title="Road Type vs Accident Count")
#Single carriageway maksimum kaza sayısına sahip sonrasında Dual carriageway takip ediyor

In [ ]:
df["Hour"] = df["Time"].str[:2]
df["Hour"] = pd.to_numeric(df["Hour"],errors="coerce")

df.groupby("Hour")["Accident_Index"].nunique().plot(kind="line",marker='o')
plt.title("Hour of Day vs Accident Count")
plt.xlabel("Hour of Day")
plt.ylabel("Number of Accidents")

plt.xticks(range(0,24))

plt.grid(True)
plt.tight_layout()
plt.show()

# Sabah 8 ve akşam 6 da maksimum kaza sayısı.Trafik yoğunluğunun olduğu saatlerde 
# maksimum kaza oluyor

In [ ]:
# Hava durumuna göre kaza oranı
df.groupby("Weather_Conditions")["Accident_Index"].nunique().sort_values().plot(kind="barh", title="Weather vs Accident Count")
# Normal havalarda (Fine without high winds) en fazla kaza sayısı gözüküyor
# Ardından yağmurlu havalar geliyor (Raining without high winds)

In [ ]:
# Kırsal/Kentsel karşılaştırması

df.groupby("Urban_or_Rural_Area")["Accident_Index"].nunique().plot(kind="bar", title="Urban vs Rural Accidents")

#Kırsalda daha az kaza olabilir ama şiddeti daha yüksek olabilir (bunu Accident_Severity ile ilişkilendirirsin) 

#Şehirde kaza daha fazla

In [ ]:
pd.crosstab(df["Speed_limit"], df["Accident_Severity"], normalize='index').plot(kind="bar", stacked=True)
# En ölümcül kazalar 10 hızında olmuş

In [ ]:
pd.crosstab(df["Urban_or_Rural_Area"], df["Accident_Severity"], normalize='index')


* Kırsal alanlarda ölümcül kaza oranı yaklaşık 3 kat daha fazla (2.3%).
→ Bu, kırsalda hız sınırlarının daha yüksek ve acil yardım ulaşımının daha geç olabileceğini düşündürebilir.

* Kentsel alanlarda kazalar daha çok hafif düzeyde (%86.9)
→ Belki yoğun ama düşük hızlı trafik sebebiyle.

* Ciddi kaza oranı da kırsalda biraz daha yüksek (%15.9)



## Problem1: Trafik akışının değişmesi kazaları nasıl etkiler ?
* Hız sınırı 30 iken kaza sayısı maksimum noktadır 0-20 arası minimal 30-70 arası daha fazla kaza vardır
* Yol tipinin tek yönlü şerit olması kaza olma olasılığını arttırır.Kazaların çoğunluğu bu yol tipinde görülmüştür.Sonrasında en fazla kaza sayısı çift yönlü yolda görülmüştür.
* Trafik yoğunluğunun artması kaza sayısını arttırır.Maksimum kaza sayısı sabah 8 civarı ile akşam 6 civarında görülmüştür.
* Şehirde kırsal alana göre daha fazla kaza görülmesine karşın kırsal alandaki kazalar genellikle daha ölümcül ve ciddi kazalardır.

## Problem 2: Kaza Oranlarını ne arttırır?

In [ ]:
#Gece mi gündüz mü daha riskli?
df.groupby('Light_Conditions')['Accident_Index'].nunique().plot(kind='bar',title = 'Accidents according to Light conditions ')

# Grafik incelendiğinde maksimum kaza sayısı günışığında yapılmış görülüyor.En fazla trafik yoğunluğu bu saatlerde olduğu için olabilir
# Sonrasında 200.000 civarı kaza ile karanlıkta(sokak lambası ile aydınlatılmış) yapılmış görünüyor.

In [ ]:
# Gece yapılan kazalar mı daha ölümcül yoksa gündüz yapılan kazalar mı ?
pd.crosstab(df['Light_Conditions'],df['Accident_Severity'],normalize='index')
# En yüksek ölümcül kaza oranı -> Darkeness: No street lighting.
# Aydınlatma varsa ama çalışmıyorsa ölümcül kaza oranı yine artıyor
# En güvenli ortam:"Daylight: Street light present" ve "Street lighting unknown" → Ölümcül kaza oranları düşük (%1.04 ve %1.18)



In [ ]:
#Hız limiti arttıkça ölümcül kaza oranı da artıyor mu?
pd.crosstab(df['Speed_limit'],df['Accident_Severity'],normalize='index')

In [ ]:
df['Speed_limit'].value_counts()
# Dikkat !! : 0 ve 15 km/s hız sınırlarında kaza sayısı çok düşük 
# Bu nedenle bu hızlara ait ölümcül oranlar güvenilir değil ve analizlerde dikkate alınmamalı.
# Anlamlı hız limitleri : 20,30,40,50,60,70

In [ ]:
df_filtered = df[df['Speed_limit'] >= 20]
# 20 altındaki hız limitleri çıkarıldı

In [ ]:
pd.crosstab(df_filtered['Speed_limit'],df['Accident_Severity'],normalize='index')
# Hız arttıkça ölümcül kaza oranı artıyor (özellikle 60 km/s’te zirve)
# Ciddi kaza(2) oranı da 60 km/s doğru artıyor
# Düşük hızlarda hafif kaza oranı daha yüksek

In [ ]:
# Haftanın günlerine göre kaza sayısı
df.groupby('Day_of_Week')['Accident_Index'].nunique()
# En çok kaza cumartesi günü görülmüş,pazar ve pazartesi ise daha az kaza görülmüş
# Cumartesi günlerindeki yoğun araç trafiği daha fazla kazaya sebebiyet vermiş olabilir.

In [ ]:
# Haftanın günlerine göre kaza şiddeti
pd.crosstab(df['Day_of_Week'],df['Accident_Severity'],normalize='index')
# En ölümcül kazalar pazar ve pazartesi görülmüş.Pazar günleri dönüş yorgunluğu,pazartesi stresi ölümcül kazaların sebepleri arasında yer alabilir
# Hafif kazaların oranı en çok Çarşamba ve Perşembe günlerinde yüksek.

In [ ]:
df.groupby('Number_of_Vehicles')['Accident_Index'].nunique()

In [ ]:
df_filtered = df[df['Number_of_Vehicles']<=7]
pd.crosstab(df_filtered['Number_of_Vehicles'],df['Accident_Severity'],normalize='index')
# Zincir kazalarda daha ölümcül sonuçlar ortaya çıkmış. 2 aracın sebep olduğu kazalarda ise ölümcül kazalar az.Daha çok hafif kazalar var.

Kazaya karışan araç sayısı arttıkça ölümcül kaza oranı da artış göstermektedir. Özellikle 6 ve 7 araçlı kazalarda ölümcül oran %4.4 seviyelerine çıkarak zirveye ulaşmıştır. En düşük ölüm riski %0.87 ile iki araçlı kazalarda görülmektedir. Buna karşın, tek araçlı kazalarda ciddi yaralanma oranı (%19.6) en yüksek düzeydedir. Bu durum, çoklu çarpışmalarda ölüm riskinin, tek araçlı kazalarda ise ağır yaralanma riskinin daha fazla olduğunu göstermektedir.

## Problem 3 : Zaman içinde yıllara göre kaza oranları nasıl değişmiş? Gelecek yıllar için tahmin yapılabilir mi?

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)

In [ ]:
df['Date'].head(10)


In [ ]:
annual_accidents = df.groupby('Year')['Accident_Index'].nunique()
annual_accidents
# annual_accidents.plot(kind='line',title='Yıllara göre kaza sayısı',marker ='o')
#2006'dan 2013 e kadar kaza sayılarında istikrarlı bir azalış var.2013-2014 arası kaza sayılarında 9000 kadarlık bir artış var


In [ ]:
# 100.000 kazadan kaç tanesi ölümcül?
severity_per_year = df.groupby(['Year', 'Accident_Severity'])['Accident_Index'].count().unstack()
severity_ratio = severity_per_year.div(severity_per_year.sum(axis=1), axis=0)
severity_ratio.plot(kind='line', title='Yıllara Göre Kaza Şiddeti Oranları')

In [ ]:
X = np.array(annual_accidents.index).reshape(-1,1) # yıllar
y = np.array(annual_accidents.values) # kazasayıları

model = LinearRegression()
model.fit(X, y)

future_years = np.array([2015,2016,2017,2018]).reshape(-1,1)
predictions = model.predict(future_years)

for year, pred in zip(future_years.flatten(), predictions):
    print(f"{year} için tahmin edilen kaza sayısı: {int(pred):,}")

**2005–2013 yılları arasında trafik kazalarında düzenli bir azalma eğilimi gözlenmiştir. 2014’teki artış geçici bir sapma olarak değerlendirildiğinde, Lineer Regresyon modeli 2015 sonrası için mantıklı bir düşüş trendi tahmin etmiştir. Ancak bu modelin sınırlı olması nedeniyle ani dalgalanmaları öngörme kabiliyeti düşüktür.**

## Problem4: Kırsal ve kentsel alanlar nasıl farklılaştı?

Kentsel alanlarda kazaların büyük çoğunluğu hafif yaralanma ile sonuçlanırken, kırsal alanlarda ölümcül kaza oranı yaklaşık 3 kat daha fazladır. Bu fark, kırsal bölgelerdeki yüksek hız limitleri, az sayıda trafik kontrol noktası ve geç müdahale süresiyle açıklanabilir. Ayrıca kırsalda gece görüş koşulları ve yol yapısı da kazaların ciddiyetini artırabilir

In [ ]:
sns.countplot(x='Accident_Severity',hue='Urban_or_Rural_Area',data=df)
plt.title('Kırsal ve Kentsel Alana Göre Kaza şiddeti')
plt.xlabel("Kaza Şiddeti (1=Ölümcül, 2=Ciddi, 3=Hafif)")
plt.xticks([0,1,2], ['Fatal', 'Serious', 'Slight'])
plt.legend(title="Area (1=Urban, 2=Rural)")
plt.show()

# GENEL TRAFİK KAZALARI ANALİZ RAPORU
### 1) Trafik Akışının Değişmesi Kazaları Nasıl Etkiler?
* Trafik akışını etkileyen faktörler arasında yol tipi (Road_Type), hız limiti (Speed_limit), günün saati (Time), hava koşulları (Weather_Conditions) ve yerleşim türü (Urban_or_Rural_Area) analiz edilmiştir.

* Yüksek hızlı yollar (örneğin 60–70 km/h) genellikle kırsal alanlarda yer almakta ve bu bölgelerde ölümcül kaza oranı belirgin şekilde yüksektir.

* Gece saatlerinde ve olumsuz hava koşullarında kaza oranları artmakta, özellikle ışıklandırmanın yetersiz olduğu bölgelerde ciddi kazalar öne çıkmaktadır.

* Kentsel alanlarda trafik akışı daha yoğun olmasına rağmen, kazaların çoğu hafif yaralanmalıdır. Bu, hız limitlerinin daha düşük olmasından kaynaklanabilir.

### 2) Kaza Oranlarını Ne Artırır?
Kaza şiddetini ve sayısını etkileyen temel faktörler şunlardır:

* Hız Limiti:
Ölümcül kazaların oranı hız limiti arttıkça yükselmiştir. Örneğin 20–30 km/s sınırlarında ölümcül kaza oranı %0.5–0.6 iken, 60 km/s'te bu oran %3'ün üzerine çıkmaktadır.

* Işık Koşulları:
Karanlık ve yetersiz aydınlatılmış bölgelerde ölümcül kazalar daha fazladır.
"Darkness: No street lighting" kategorisinde ölümcül kaza oranı %4.3’tür.

* Araç Sayısı:
Çok araçlı kazalarda ciddi ve ölümcül kaza oranı artmaktadır. Ancak tek araçlı kazalarda da ölümcül oran gözle görülür biçimde yüksektir — bu genellikle kırsal alanlardaki yoldan çıkma kazalarından kaynaklanabilir.

* Günler:
Cumartesi günleri toplam kaza sayısı en yüksektir. Pazar günü ise ölümcül kaza oranı belirgin şekilde daha yüksektir.

### 3) Zaman İçinde Kaza Oranlarını Tahmin Edebilir Miyiz?
Yıllık toplam kaza sayıları 2005 yılında 128.995 iken, 2014 yılına kadar genel olarak azalarak 91.141’e düşmüştür. Bu azalan trend, Lineer Regresyon ile modellenmiş ve 2015–2018 yılları için tahmini kaza sayıları sırasıyla şu şekilde öngörülmüştür:

#### Yıl	Tahmin Edilen Kaza Sayısı
* 2015  	77.523
* 2016	    72.726
* 2017	    67.930
* 2018	    63.133

Model, geçmiş verilere uygun bir şekilde azalan bir eğilim tahmin etmiştir. Ancak, 2014’teki artış gibi ani dalgalanmaları yakalayamadığı için daha gelişmiş zaman serisi modelleri (ARIMA, Prophet) kullanılması önerilir.

### 4) Kırsal ve Kentsel Alanlar Nasıl Farklılaştı?
Kentsel alanlarda toplam kaza sayısı daha fazla, ancak kaza şiddeti genelde hafif.

Kırsal alanlarda kaza sayısı daha az olmasına rağmen, ölümcül kaza oranı yaklaşık 3 kat daha yüksek.
Bu farkın nedenleri:

1) Daha yüksek hız limitleri (60–70 km/h)

2) Daha az ışıklandırma

3) Müdahale süresinin daha uzun olması

4) Karmaşık yol yapıları (örneğin kavşaklar)

Bu sonuçlar, kırsal alanlarda trafik güvenliğine yönelik özel önlemler alınması gerektiğini göstermektedir.